<div class="row">
  <div class="column">
    <img src="./img/logo-onera.png" width="200">
  </div>
  <div class="column">
    <img src="./img/logo-ISAE_SUPAERO.png" width="200">
  </div>
</div>

# The FAST-OAD framework

One of the main differences between FAST-OAD and OpenMDAO is that FAST-OAD uses specific files as user interface, the main one being the configuration file. This configuration file enables the user to change the model that are used by simply changing the names called. As a consequence, any change in the configuration file will change the problem that we want OpenMDAO to solve. In order to do so, we simply have to give each module an ID, which will then be used in the configuration file. More information on the configuration file are available in [the custom module tutorial](../01_custom_modules/beam_problem.ipynb). This creates a intermediate interface which allows the user to not go into the code if he so wishes. 

The user is encouraged to check out the [the custom module tutorial](../01_custom_modules/beam_problem.ipynb) before finishing this one as both are complementary.

The first thing that needs to be done is to associate our module to some IDs. We will start by doing it only for the global group. To make the IDs more identifiable in this tutorial, we will have them start with "tutorial."

Compared to the models used in the [OpenMDAO notebook](04_OpenMDAO.ipynb), the only block we will need to change is the block that creates the design loop. Indeed, FAST-OAD does not yet support the affectation of the default value after the setup (which is how it was done in the [previous example](04_OpenMDAO.ipynb)), so we need to overwrite it inside the module that computes the new **MTOW**. It is important to do it because, if not done, OpenMDAO will start iterating with an MTOW of 1.0 kg (the default value in OpenMDAO) which will ultimately cause the process to crash, as illustrated in the [previous notebook](04_OpenMDAO.ipynb). The new building block is [here](modules/FAST-OAD/performance/sub_components/compute_new_mtow.py), the only change being that when adding the output, we give a default value at 500.0 kg which is what we did in the previous examples.

Let's check that the modules is correctly registered. We will use the same tool as in [the custom module tutorial](../01_custom_modules/beam_problem.ipynb). You will notice that six IDs are listed, the reasoning will be explained further in the notebook.

In [ ]:
import os.path as pth
import fastoad.api as oad
import numpy as np

CUSTOM_MODULES_FOLDER_PATH = "./modules"

oad.list_modules(CUSTOM_MODULES_FOLDER_PATH)

We can indeed see that the ID for the global group was properly registered. Let's now move on to comparing FAST-OAD to OpenMDAO.

## Using FAST-OAD

### Global MDA

Let's setup a configuration file that matches what is done in the [previous notebook](04_OpenMDAO.ipynb). To do that and as explained in the [the custom module tutorial](../03_custom_modules) we will simply fill the configuration file with the ID corresponding to our global group.

For the sake of comparison, the reader should keep in mind that writing this in the configuration file :

>```yaml
>model:
>
>  global:
>    id: "tutorial.fast_oad.global"
>```

is the equivalent in pure OpenMDAO of doing this :

>```python
prob.model.add_subsystem(name="global", subsys=UpdateMTOW(), promotes=["*"])
>```

The problem is automatically created when using the configuration file, so the only requirement is to say which model we want to use.

The configuration file we will use for the first MDA is located [here](workdir/MDA_global_group/mda_global_group.yml)

In [ ]:
CONFIGURATION_FILE_GLOBAL_MDA = "./working_folder/MDA_global_group/mda_global_group.yml"

Now that our configuration file is set, we can create the input data file which corresponds to the problem inputs that we will need to fill in order for FAST-OAD to have all the information it needs to solve the problem. FAST-OAD allows to generate an input data file from the problem definition in the configuration file. By default, it will contain the default values, but the generation process can also use values from a source data file.

A handy analogy for how the files work is presented below.

<div class="column">
    <img src="./img/handy_analogy.PNG" width="1000">
</div>

We will present both way to do it, but using the source data file is the prefered way in FAST-OAD simply because we can fill an .xml with all the potential value we might need and reuse it both for the MDA and the MDO. In the following example for instance, we won't have to re-fill a new input data file each time and we won't have to delete the aspect_ratio for the MDO as it will simply not be transfered in the input data file when necessary.

The source data file we will reuse for every problem is [here](data/source_file.xml).

In [ ]:
# To generate an empty configuration file. The location of the file just generated is specified inside the configuration
# file. Here we decided to put it next to the configuration file.
# UNCOMMENT NEXT LINE IF YOU DARE TO TRY THIS METHOD.
# oad.generate_inputs(CONFIGURATION_FILE_GLOBAL_MDA, overwrite=True)

# The command to generate the input data file based on the source is very similar, we only need to provide the path to the
# source data file and the newly generated input data file will be placed at the location specified inside the configuration file.
SOURCE_FILE = "./data/source_file.xml"
oad.generate_inputs(CONFIGURATION_FILE_GLOBAL_MDA, SOURCE_FILE, overwrite=True)

Now that we have generated the input and since we have our "recipe", we can launch the problem by simply executing the next cell. FAST-OAD will add the solver according to the content of the configuration file, set up the problem and run it. It is important to generate the inputs before launching the problem.

In [ ]:
mda_global = oad.evaluate_problem(CONFIGURATION_FILE_GLOBAL_MDA, overwrite=True)

As before, we can access any value by its name, or this time we can also look at the output file which was created based on the last value computed by the code and is located where indicated in the configuration file. In our case, it should be [here](workdir/MDA_global_group/problem_outputs.xml). Alternatively, we could also make use of the variable viewer from FAST-OAD which provides an easy way to sweep through the variables of the problem.

In [ ]:
print("MTOW:", float(np.round(mda_global.get_val("mtow", units="kg"), 1)), "kg")
print("Mission fuel:", float(np.round(mda_global.get_val("mission_fuel", units="kg"), 1)), "kg")
print("Wing mass:", float(np.round(mda_global.get_val("wing_mass", units="kg"), 1)), "kg")

In [ ]:
oad.variable_viewer(mda_global.output_file_path)

As expected, we have precisely the same results since FAST-OAD is an added layer to OpenMDAO. 

We made the choice in this part to only register the global group as opposed to, for example, registering the subgroups. This raises a question regarding the level of detail and modularity at which to register models. It is absolutely possible to register every single block that was coded in order to have more control on what is computed in each run. This would mean that we would have to give every single base bricks an ID. The consequence would be that the configuration file will be very long as we would need to add every single ID. 

In FAST-OAD and in the next example the choice was made to put the IDs at the discipline level, which means that we have one ID for the geometry, one ID for the aerodynamics, ... This explains why there was 6 IDs in total earlier (5 for the disciplines in addition to the global group). The basic block will almost all remain the same so we will reuse them directly and just rewrite the discipline block and the UpdateMTOW block. This is done in the [FAST-OAD folder](a "02_explanation/modules/FAST-OAD"). 

We will now do the exercice of calling the disciplines one by one instead of calling a group that contains the manual addition of each of the modules. This is also a good way to have a gentle introduction to the type of configuration file which can be found in FAST-OAD. The results should be exactly the same.

### MDA discipline by discipline

As said before, instead of calling the global group, we will call, in our new configuration file, the disciplines one by one and in the same order as they were added in the global group. This is done in [this configuration file](workdir/MDA_discipline/mda_discipline.yml). As before, we will generate the input data file based on the source data file and simply run the problem. We could also use the input data file from the previous MDA since the inputs should be the same (we use the same modules simply called differently).

In [ ]:
CONFIGURATION_FILE_DISCIPLINE_MDA = "./working_folder/MDA_discipline/mda_discipline.yml"

# Generate the input data file based on configuration and source data file
oad.generate_inputs(CONFIGURATION_FILE_DISCIPLINE_MDA, SOURCE_FILE, overwrite=True)

# Run the problem
mda_discipline = oad.evaluate_problem(CONFIGURATION_FILE_DISCIPLINE_MDA, overwrite=True)

In [ ]:
print("MTOW:", float(np.round(mda_discipline.get_val("mtow", units="kg"), 1)), "kg")
print("Mission fuel:", float(np.round(mda_discipline.get_val("mission_fuel", units="kg"), 1)), "kg")
print("Wing mass:", float(np.round(mda_discipline.get_val("wing_mass", units="kg"), 1)), "kg")

Then again we have the same results as before. Let's now move on to the declaration of an MDO in FAST-OAD

## MDO(s)

In order to declare an MDO in FAST-OAD, we have to, same as in the previous example, declare which variables are gonna be the design variables, how they are bound and what is the objective we want to minimize. This time however, everything is done at the bottom of the configuration file. Let us take again the example of finding the aspect ratio that gives the best **MTOW**. 

Declaring the aspect ratio as the design variable as well as giving it its bounds is done like so, under the optimization segment of the yml.

>```yml
>design_variables:
>    - name: aspect_ratio
>      lower: 2.0
>      upper: 20.0
>```

Declaring the objective is done like this:

>```yml
>objective:
>    - name: mtow
>```

The final configuration file is written [here](workdir/MDO_MTOW/mdo_mtow.yml). As for an MDA, we need to generate an input data file, which we will do using the source data file. 

Before launching the MDO we can make a small remark. In this MDO we are using the same modules as in the MDA. This time however we let the aspect ratio be free within two bounds, this means that it is no longer a required input as it will be handled by the optimizer. We can check that this is indeed the case by checking that the aspect_ratio is absent from the MDO input data file.

For the MDO, FAST-OAD provides the user with an optimization viewer, which very similar to the variable viewer. Its use is shown here

In [ ]:
CONFIGURATION_FILE_MDO_MTOW = "./working_folder/MDO_MTOW/mdo_mtow.yml"

# Generate the input data file based on configuration and source data file
oad.generate_inputs(CONFIGURATION_FILE_MDO_MTOW, SOURCE_FILE, overwrite=True)

oad.optimization_viewer(CONFIGURATION_FILE_MDO_MTOW)

The optimization is then launched by doing the following :

In [ ]:
mdo_mtow = oad.optimize_problem(CONFIGURATION_FILE_MDO_MTOW, overwrite=True)

The configuration viewer can be reused after the optimisation for a convenient summary of the results, or, for a more detailed one, we can, as before, either call each variable, fetch the output file or use the variable_viewer

In [ ]:
oad.optimization_viewer(CONFIGURATION_FILE_MDO_MTOW)

In [ ]:
oad.variable_viewer(mdo_mtow.output_file_path)

What we can observe is that the same results were obtained as before.

Checking that the optimisation on the mission fuel gives the same result is left to the user as well as the optimization on two parameters. Adding a new line with the cruise speed should be the only change required to get things working. The results should also be the same as before.

This concludes this serie of tutorials on how to go from the definition of a pure python problem into a FAST-OAD problem. Any criticism and idea for improvement is welcome.